In [1]:
import pandas as pd
import string
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
df1 = pd.read_csv('1_Rating.csv')
df2 = pd.read_csv('2_Rating.csv')
df3 = pd.read_csv('3_Rating.csv')
df4 = pd.read_csv('4_Rating.csv')
df5 = pd.read_csv('5_Rating.csv')

df1.drop(columns=['Unnamed: 0'], inplace = True)
df2.drop(columns=['Unnamed: 0'], inplace = True)
df3.drop(columns=['Unnamed: 0'], inplace = True)
df4.drop(columns=['Unnamed: 0'], inplace = True)
df5.drop(columns=['Unnamed: 0'], inplace = True)

In [3]:
one_star_reviews = df1['Text'].tolist()
two_star_reviews = df2['Text'].tolist()
three_star_reviews = df3['Text'].tolist()
four_star_reviews = df4['Text'].tolist()
five_star_reviews = df5['Text'].tolist()

In [4]:
one_star_reviews[50]

'The Sugar Factory is known for it\'s delicious drinks and some of the worst service in town.\n\nSeriously, I\'ve had a problem with the Sugar Factory on the strip with customer service, and after visiting this location for the first time the other day, my hopes of better service crumbled, it\'s even worse here.\n\nWhile the hostess was great and quick about seating us, we sat there for about 12 minutes before anyone even came over. Not only was the waitress slow, she seemed confused as to what tables she was suppose to be waiting. A moment after she left, another waiter came to take our order once again, and was confused that we had ordered with someone else already.\n\nI usually get the veggie burger from Sugar Factory, not only is it really the only acceptable vegetarian dish on the menu, it\'s just delicious. I don\'t know whether they changed the recipe or the chef is an idiot, but the once signature purple bun was replaced with a regular white bun. Bummer. :(\n\nFinishing touch t

# Need to clean up the text in the reviews as I tokenize so my generator doesn't produce something like '\n'.

In [5]:
def cleanin_reviews(review):
    # Many reviews have \n characters through my scrape, so I need to replace those with empty spaces
    review = review.replace('\n\n', ' ')
    # Splitting on spaces to tokenize
    tokens = review.split()
    # Getting rid of punctuation because 'word.' will be considered different from 'word'
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table) for word in tokens]
    # Simplifying my vocabulary to only include words and not smiley faces/other datatypes
    tokens = [word for word in tokens if word.isalpha()]
    
    tokens = [word.lower() for word in tokens]
    return tokens

In [6]:
def tokenizer(star):
    all_tokens = []

    for reviews in star:
        all_tokens.append(cleanin_reviews(reviews))
    
    # The above makes a list of lists, but I need to have one giant list of tokens
    flat_list = [item for sublist in all_tokens for item in sublist]
    return flat_list

# One Star Reviews

In [7]:
one_stars = tokenizer(one_star_reviews)

In [8]:
print('Total Tokens: %d' % len(one_stars))
print('Unique Tokens: %d' % len(set(one_stars)))

Total Tokens: 1370411
Unique Tokens: 37674


# Two Star Reviews

In [9]:
two_stars = tokenizer(two_star_reviews)

In [10]:
print('Total Tokens: %d' % len(two_stars))
print('Unique Tokens: %d' % len(set(two_stars)))

Total Tokens: 1485599
Unique Tokens: 40776


# Three Star Reviews

In [11]:
three_stars = tokenizer(three_star_reviews)

In [12]:
print('Total Tokens: %d' % len(three_stars))
print('Unique Tokens: %d' % len(set(three_stars)))

Total Tokens: 3181354
Unique Tokens: 61030


# Four Star Reviews

The lower total tokens count represents the smaller datasets for the review set. If the model ends up not being as robust as the others, I will obtain additional datapoints.

In [13]:
four_stars = tokenizer(four_star_reviews)

In [14]:
print('Total Tokens: %d' % len(four_stars))
print('Unique Tokens: %d' % len(set(four_stars)))

Total Tokens: 489058
Unique Tokens: 30313


# Five Star Reviews

The lower total tokens count represents the smaller datasets for the review set. If the model ends up not being as robust as the others, I will obtain additional datapoints.

In [15]:
five_stars = tokenizer(five_star_reviews)

In [16]:
print('Total Tokens: %d' % len(five_stars))
print('Unique Tokens: %d' % len(set(five_stars)))

Total Tokens: 670280
Unique Tokens: 37641


In [20]:
def sequence_of_tokens(stars):
    length = 5 + 1
    sequences = list()
    for i in range(length, len(stars)):
        seq = stars[i-length:i]

        line = ' '.join(seq)

        sequences.append(line)
    print('Total Sequences: %d' % len(sequences))
    return sequences

In [21]:
one_star = sequence_of_tokens(one_stars)

Total Sequences: 1370405


In [22]:
two_star = sequence_of_tokens(two_stars)

Total Sequences: 1485593


In [23]:
three_star = sequence_of_tokens(three_stars)

Total Sequences: 3181348


In [24]:
four_star = sequence_of_tokens(four_stars)

Total Sequences: 489052


In [25]:
five_star = sequence_of_tokens(five_stars)

Total Sequences: 670274


# Need to save these different sequences down to train my model

In [43]:
def saver(lines, name):
    data = '\n'.join(lines)
    file = open(name, 'w')
    file.write(data)
    file.close()

In [44]:
saver(one_star, 'One_Star.txt')
saver(two_star, 'Two_Star.txt')
saver(three_star, 'Three_star.txt')
saver(four_star, 'Four_Star.txt')
saver(five_star, 'Five_Star.txt')